# Data loading an prep

In [2]:
from langchain.docstore.document import Document
from datasets import load_dataset
import pandas as pd

df = pd.read_csv(
    "./data/annonces_aveccaracteristiques_preprocesse.csv"
)

# Define a helper function to convert a dataset entry to a Document
# This object has two main components: page_content and metadata.

def entry_to_document(entry):
    """
    Convert a single dataset entry to a Document object. every dataset entry is converted into a Document object in a consistent manner. This consistency is crucial for downstream processing...
    Args:
        entry (dict): A single entry from the dataset.
        
    page_content: contains a formatted string that includes all the relevant information from the dataset entry
    """
    return Document(
        page_content="\n".join([
            f"ID: {entry['ID']}",
            f"Titre: {entry['Titre']}",
            f"Prix: {entry['Prix']}",
            f"Localisation: {entry['Localisation']}",
            f"Surface: {entry['Surface']}",
            f"Pièces: {entry['Pièces']}",
            f"Chambres: {entry['Chambres']}",
            f"DPE: {entry['DPE']}",
            f"Description: {entry['Description']}",
            f"Date de publication: {entry['Date de publication']}",
            f"Agence: {entry['Agence']}",
            f"""Logo de l'agence: {entry["Logo de l'agence"]}""",
            f"Images: {entry['Images']}",
            f"Lien annonce: {entry['Lien annonce']}",
            f"Lien page source: {entry['Lien page source']}",
            f"Caracteristiques: {entry['Caracteristiques']}"
        ]),
        metadata={"ID": entry["ID"]}
    )

# Convert dataset entries into Document objects
docs = [ entry_to_document(row) for _, row in df.iterrows() ]

In [8]:
docs

[Document(metadata={'ID': 'xxuijv'}, page_content="ID: xxuijv\nTitre: Appartement • 77\xa0m² • 4\xa0pièces • 3\xa0chambres\nPrix: 220000.0\nLocalisation: Villeurbanne (69100)\nSurface: 77.0\nPièces: 4.0\nChambres: 3.0\nDPE: d\nDescription: T4 Villeurbanne Perralière garage possible\nT4 74m² Villeurbanne\nJe vous propose en exclusivité cet appartement :\nIl se compose d'une entrée, d'un séjour, d'une cuisine, 3 chambres, une salle de bain et des toilettes. toutes les pièces à part la cuisine donnent sur le balcon filant. L'appartement est entièrement climatisé.\nune cave complète l'appartement et un garage box en sous-sol est possible pour 15 000 € supplémentaire.\nSitué à la Perralière, vous trouverez tous les commerces à proximité. l'arrêt de métro Flachet à moins de 5 minutes à pieds ainsi que le tram Grand clément à peu près à la même distance. De plus des pistes cyclables sont accessibles facilement.\nSitué dans un grand parc avec terrain de tennis et jeux de pétanques, vous aurez 

# Retriever

# Retriever
## Semantic Search Models
 
The following models have been specifically trained for Semantic Search: Given a question / search query, these models are able to find relevant text passages. For more details, see Usage > Semantic Search.

Description:	All-round model tuned for many use-cases. Trained on a large and diverse dataset of over 1 billion training pairs.
Base Model:	microsoft/mpnet-base
Max Sequence Length:	384
Dimensions:	768
Normalized Embeddings:	true
Suitable Score Functions:	dot-product (util.dot_score), cosine-similarity (util.cos_sim), euclidean distance
Size:	420 MB
Pooling:	Mean Pooling
Training Data:	1B+ training pairs. For details, see model card.
Model Card:	https://huggingface.co/sentence-transformers/all-mpnet-base-v2

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
from langchain.tools import Tool
from langchain.docstore.document import Document

# Extract just the text strings before encoding!
texts = [doc.page_content for doc in docs]
texts

# loading pretrained SentenceTransformer model
embed_model = SentenceTransformer("all-mpnet-base-v2")

# return a (N_docs, dim) numpy array
doc_embeddings: np.ndarray = embed_model.encode(texts, convert_to_numpy=True)
doc_embeddings

#Then will have to use an indexing structure to store the embeddings and retrieve them efficiently with some similarity search algorithm.

# FAISS Alternative

In [4]:
from langchain.vectorstores import FAISS
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.tools import Tool

# Wrap your sentence‐transformer in a LangChain Embeddings object
embedding_model = SentenceTransformerEmbeddings(model_name="all-mpnet-base-v2")

# FAISS index directly from docs. By default it builds a flat L2 (Euclidean)‑based index.
vectorstore = FAISS.from_documents(docs, embedding_model)

# Turn that into a retriever. "k" is the number of documents to return
semantic_retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# Now we can use the retriever to get relevant documents for a given query
def extract_text_semantic(query: str) -> str:
    results = semantic_retriever.get_relevant_documents(query)
    if not results:
        return "No matching information found..."
    return "\n\n".join(doc.page_content for doc in results)

# tool for extracting info from our data 
immo_info_retriever_tool = Tool(
    name="immo_info_retriever_tool",
    func=extract_text_semantic,
    description="Retrieves detailed information about apartments based on a query. "
                "Useful for answering questions about apartment features, prices, and locations."
)

# Adding the similarity measures

In [27]:
from langchain.vectorstores import FAISS
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.tools import Tool

# Wrap your sentence‐transformer in a LangChain Embeddings object
embedding_model = SentenceTransformerEmbeddings(model_name="all-mpnet-base-v2")

# FAISS index directly from docs. By default it builds a flat L2 (Euclidean)‑based index.
vectorstore = FAISS.from_documents(docs, embedding_model)

def extract_text_semantic(query: str) -> str:
    # Retrieve documents and scores using L2 distance
    results = vectorstore.similarity_search_with_score(query, k=5)
    if not results:
        return "No matching information found..."
    
    # Format results with content and scores
    output = []
    for doc, score in results:
        output.append(f"{doc.page_content}\nSimilarity Score (L2 Distance): {score:.2f}\n")
    return "\n\n".join(output)

# Create the tool
immo_info_retriever_tool = Tool(
    name="immo_info_retriever_tool",
    func=extract_text_semantic,
    description="Retrieves apartment details with similarity scores. Use for questions about features, prices, or locations."
)

response = immo_info_retriever_tool.run("Appartements de plus de 61 m2 à Lyon centre")
print(response)

ID: xxseyu
Titre: Appartement • 54,42 m² • 3 pièces • 2 chambres
Prix: 169600.0
Localisation: Lyon 5ème (69005)
Surface: 54.42
Pièces: 3.0
Chambres: 2.0
DPE: e
Description: À VENDRE : Charmant appartement de 54 m² à Lyon.
Découvrez cet appartement lumineux et fonctionnel idéalement situé dans la ville de Lyon. Avec ses 54 m², il offre un cadre de vie agréable et pratique, parfait pour un couple ou une petite famille.
Les points forts de ce bien :
- Entrée accueillante pour plus de confort.
- Séjour lumineux ouvrant sur un balcon idéal pour vos moments de détente.
- Cuisine meublée et fonctionnelle, prête à l'emploi.
- Deux chambres offrant calme et sérénité.
- Salle de douche moderne et WC indépendant.
- Une cave privative pour vos besoins de rangement
Pourquoi choisir cet appartement ?
Emplacement stratégique : Proche des commodités, transports et écoles.
Un espace optimisé et prêt à accueillir vos projets.
Une belle opportunité pour un premier achat ou un investissement.
Charges de c

Try with fr-camembert-base as we are only dealing with fr 

In [26]:
from langchain.vectorstores import FAISS
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.tools import Tool

# Wrap your sentence‐transformer in a LangChain Embeddings object
embedding_model = SentenceTransformerEmbeddings(model_name="dangvantuan/sentence-camembert-base")

# FAISS index directly from docs. By default it builds a flat L2 (Euclidean)‑based index.
vectorstore = FAISS.from_documents(docs, embedding_model)

def extract_text_semantic(query: str) -> str:
    # Retrieve documents and scores using L2 distance
    results = vectorstore.similarity_search_with_score(query, k=5)
    if not results:
        return "No matching information found..."
    
    # Format results with content and scores
    output = []
    for doc, score in results:
        output.append(f"{doc.page_content}\nSimilarity Score (L2 Distance): {score:.2f}\n")
    return "\n\n".join(output)

# Create the tool
immo_info_retriever_tool = Tool(
    name="immo_info_retriever_tool",
    func=extract_text_semantic,
    description="Retrieves apartment details with similarity scores. Use for questions about features, prices, or locations."
)

response = immo_info_retriever_tool.run("Appartements de plus de 61 m2 à Lyon centre")
print(response)

ID: xwyumm
Titre: Appartement • 61 m² • 3 pièces • 2 chambres
Prix: 285000.0
Localisation: Lyon 7ème (69007)
Surface: 61.0
Pièces: 3.0
Chambres: 2.0
DPE: e
Description: EN EXCLUSIVITÉ,
Orpi vous propose dans un quartier recherché, près des universités, situé au plein coeur de Lyon 7 quai du Rhône un appartement au sein d'une petite copropriété un appartement de 62 m² environ.
Il est à reconfigurer en T3 afin d'avoir un spacieux salon, 2 chambres aux calme, SDB et WC.
Parfait pour un 1er achat
Idéalement situé au cœur d'un dynamique quartier, avec commerces, restaurants, métro et tramway à proximité mais également à moins de 10 minutes de la place Bellecour
Vous serez séduit par sa localisation et ses beaux volumes
1 cave et 1 grenier complète ce bien
A voir rapidement
Contactez-nous pour plus d'informations ou pour planifier une visite dès aujourd'hui.
Référence agence : 5091
Date de publication: 2024-10-17
Agence: Immobilière Du Parc
Logo de l'agence: https://photo.superimmo.com/cpmbb

We have higher distances with dangvantuan/sentence-camembert-base so perhaps we have poor semantic matches....

We could try several embeddings and see how they perform based on one or more test queries. Refering to .gitignored (embedding_comparateur.ipynb)

# Agent Construction